# 1. Importing Libraries

In [1]:
!pip install --upgrade opencv-python


  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/ec/6c/fab8113424af5049f85717e8e527ca3773299a3c6b02506e66436e19874f/opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.8 MB 2.8 MB/s eta 0:00:14
   ---------------------------------------- 0.3/38.8 MB 2.6 MB/s eta 0:00:15
    --------------------------------------- 0.5/38.8 MB 3.7 MB/s eta 0:00:11
    --------------------------------------- 0.6/38.8 MB 3.0 MB/s eta 0:00:13
    --------------------------------------- 0.7/38.8 MB 3.0 MB/s eta 0:00:13
    --------------------------------------- 0.8/38.8 MB 2.7 MB/s eta 0:00:14
    --------------------------------------- 0.9/38.8 MB 2.7 MB/s eta 0:00:14
   - -------------------------------------- 1.2/38.8 MB 3.0 MB/s eta 0:00:13
   - -----------------

ERROR: Could not install packages due to an OSError: [Errno 28] No space left on device



In [2]:
!pip install --upgrade cvlib

  Using cached cvlib-0.2.7-py3-none-any.whl
  Using cached progressbar-2.5-py3-none-any.whl
  Using cached imutils-0.5.4-py3-none-any.whl


In [3]:
import cv2
import cvlib
import numpy as np
import os
import matplotlib.pyplot as plt


# 2. Video to frames

In [4]:
# path of video
vidcap = cv2.VideoCapture(r"C:\Users\offic\Downloads\Desktop files\jupyter\inventory.mp4")
# vidcap = cv2.VideoCapture(r'C:\Users\offic\Downloads\Desktop files\jupyter\other videos\ghost.mp4')
success, frame = vidcap.read()
success 
count = 0

while vidcap.isOpened():
    success, frame = vidcap.read()
    if success:
        cv2.imwrite(r"C:\Users\offic\Downloads\Desktop files\jupyter\Inventory_frames\%d.png" %count, frame)
#         cv2.imwrite(r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\ghost_frames\%d.png" %count, frame)
        count+=1
    else:
        break

# 3. Change Detection using Background Removal technique (Background Substraction)

In [3]:
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\ghost_frames"
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\Theft_frames"
imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\Inventory_frames"

# Background subtraction is performed using OpenCV's `createBackgroundSubtractorMOG2()` to identify moving objects in each frame.
fgModel = cv2.createBackgroundSubtractorMOG2() 
for imName in os.listdir(imPath):
    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))
    
#     The resulting binary mask (`fgmask`) is combined with the original frame to visualize the detected changes.
    fgmask = fgModel.apply(frame)
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask, fgmask
    F2 = np.hstack((frame,F)) 
#     looped through all frames in the specified directory.

    cv2.imshow('Display',F2)
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
cv2.destroyAllWindows()


# 4. Denoising using Morphology

In [4]:
#Morphological operations (opening) are applied to the binary mask (`fgmask`) 
#to remove noise and small unwanted blobs from the detected foreground.

# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\ghost_frames"
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\Theft_frames"
imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\Inventory_frames"
fgModel = cv2.createBackgroundSubtractorMOG2()
for imName in os.listdir(imPath):
    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask, fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
cv2.destroyAllWindows()


# 5. Connected Components

In [5]:
def f_keepLargeComponents(I,th):
    R = np.zeros(I.shape)<0
    unique_labels = np.unique(I.flatten())
    for label in unique_labels:
        if label == 0:
            pass
        else:
            I2 = I == label
            if np.sum(I2)>th:
                R = R | I2
    return np.float32(255*R)

In [6]:
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\ghost_frames"
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\Theft_frames"
imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\Inventory_frames"
fgModel = cv2.createBackgroundSubtractorMOG2()
for imName in os.listdir(imPath):
    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = f_keepLargeComponents(labels_im,1500)
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask, fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
cv2.destroyAllWindows()


# 6. Tracking Changes

In [7]:
def f_saveSeq(C,counter,th,outputPath):
    if len(C)<th:
        pass 
    else:
        K = 1
        for frame in C:
            imName = str(counter)+'_'+ str(K)+'.jpg'
            finalPath = os.path.join(outputPath,imName)
            cv2.imwrite(finalPath,frame)
            K +=1

In [8]:
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\ghost_frames"
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\other videos\Theft_frames"
imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\Inventory_frames"
fgModel = cv2.createBackgroundSubtractorMOG2()
leastNumOfFrames = 5
idx = []
C = []
counter = 0
outputPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\0utput"
for imName in os.listdir(imPath):
    counter +=1
    frame = cv2.imread(os.path.join(imPath,imName))
    frame = cv2.resize(frame,dsize=(600,400))
    fgmask = fgModel.apply(frame)
    K_r = cv2.getStructuringElement(cv2.MORPH_RECT,(3,3))
    fgmask = cv2.morphologyEx(np.float32(fgmask),cv2.MORPH_OPEN,K_r)
    num_labels,labels_im = cv2.connectedComponents(np.array(fgmask>0,np.uint8))
    fgmask = f_keepLargeComponents(labels_im,1000)
    if np.sum(fgmask)>0:
        idx.append(counter)
        C.append(frame)
    if len(idx) >= 2 and idx[-1] > idx[-2] +1:
        f_saveSeq(C,counter,leastNumOfFrames,outputPath)
        idx = []
        C = []
        
    
    F = np.zeros(frame.shape,np.uint8)
    F[:,:,0],F[:,:,1],F[:,:,2] = fgmask, fgmask, fgmask
    F2 = np.hstack((frame,F))
    cv2.imshow('Display',F2)
    k = cv2.waitKey(10) & 0xff
    if k == 27:
        break
f_saveSeq(C,counter,leastNumOfFrames,outputPath)
cv2.destroyAllWindows()


In [11]:
def f_displaySeq(outputPath):
    for imName in os.listdir(outputPath):
        frame = cv2.imread(os.path.join(outputPath,imName))
        frame = cv2.resize(frame,dsize=(600,400))
        cv2.imshow('Display', frame)
        k = cv2.waitKey(10) & 0xff
        if k == 27:
            break
    cv2.destroyAllWindows
    


In [12]:
pth = r"C:\Users\offic\Downloads\Desktop files\jupyter\0utput"
f_displaySeq(pth)

In [13]:
# # imPath = r'C:\Users\offic\Downloads\Desktop files\jupyter\Train\Vandalism'
# imPath = r"C:\Users\offic\Downloads\Desktop files\jupyter\Inventory_frames"
# for imName in os.listdir(imPath):
#     frame = cv2.imread(os.path.join(imPath,imName))
#     frame = cv2.resize(frame,dsize=(600,400))
#     cv2.imshow('Display', frame)
#     k = cv2.waitKey(1) & 0xff
#     if k == 27:
#         break
# cv2.destroyAllWindows